<a href="https://colab.research.google.com/github/saurabhIU/Deep-Learning/blob/master/Problem_2_Speech_Denoising_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [0]:
# Dependencies
import librosa
import librosa.display as disp
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time;

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test_x_01.wav to test_x_01.wav
Saving test_x_02.wav to test_x_02.wav
Saving train_clean_male.wav to train_clean_male.wav
Saving train_dirty_male.wav to train_dirty_male.wav
User uploaded file "test_x_01.wav" with length 145284 bytes
User uploaded file "test_x_02.wav" with length 388752 bytes
User uploaded file "train_clean_male.wav" with length 2522886 bytes
User uploaded file "train_dirty_male.wav" with length 2522898 bytes


# Load Data (Training Clean and Training Noisy data)

In [0]:
s_clean, sr_clean=librosa.load('train_clean_male.wav',sr=None)
S=librosa.stft(s_clean, n_fft=1024, hop_length=512)
s_noisy, sr_noisy=librosa.load('train_dirty_male.wav',sr=None)
X=librosa.stft(s_noisy, n_fft=1024, hop_length=512)

In [0]:
S_absolute = np.abs(S)
X_absolute = np.abs(X)

# Neural Network 

In [0]:
EPOCHS = 1500
X = tf.placeholder("float", [None, 513])
Y = tf.placeholder("float", [None, 513])
def nn_model(x):
  
    layer1 = tf.layers.dense(x, 1000,activation=tf.nn.relu)
    layer2 = tf.layers.dense(layer1, 1000,activation=tf.nn.relu)
    output = tf.layers.dense(layer2, 513,activation=tf.nn.relu)
    return output

logits = nn_model(X)
cost = tf.losses.mean_squared_error(Y,logits)
optimizer = tf.train.AdamOptimizer().minimize(cost)
correct_prediction = tf.equal(tf.argmax(logits, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Training Neural Network

In [34]:
sess =  tf.Session() 
    
sess.run(tf.global_variables_initializer())
  
tic = time.time()
for i in range(EPOCHS):
    c,_,acc = sess.run([cost, optimizer, accuracy],feed_dict={X: X_absolute.T, Y: S_absolute.T})
    if i % 100 == 0:
      print(f'Epoch: {i},training accuracy:{acc * 100}')
toc = time.time()
print(f'Time taken for training is {toc-tic}')

Epoch: 0,training accuracy:0.0
Epoch: 100,training accuracy:50.14233589172363
Epoch: 200,training accuracy:52.989017963409424
Epoch: 300,training accuracy:55.18503189086914
Epoch: 400,training accuracy:55.95770478248596
Epoch: 500,training accuracy:56.93371295928955
Epoch: 600,training accuracy:58.64172577857971
Epoch: 700,training accuracy:58.80439281463623
Epoch: 800,training accuracy:60.268402099609375
Epoch: 900,training accuracy:60.390400886535645
Epoch: 1000,training accuracy:60.87840795516968
Epoch: 1100,training accuracy:61.65107488632202
Epoch: 1200,training accuracy:61.93574666976929
Epoch: 1300,training accuracy:62.38308548927307
Epoch: 1400,training accuracy:62.66775131225586
Time taken for training is 29.259649753570557


# Load test noisy signal test_01_x.wav and feed its magnitude spectra to trained network

In [0]:
test1, sr_test=librosa.load('test_x_01.wav',sr=None)
Test1=librosa.stft(test1, n_fft=1024, hop_length=513,window="hann")
Test1_absolute = np.abs(Test1)

In [68]:
Test1_absolute.shape

(513, 142)

In [69]:
test_prediction = sess.run(logits,feed_dict={X: Test1_absolute.T})
test_prediction.shape

(142, 513)

# Recover complex valued speech spectrogram of cleaned test signal

In [0]:
test1_complex = np.multiply(Test1,test_prediction.T)

In [71]:
test1_complex.shape

(513, 142)

# Recover time domain speech signal by applying inverse-STFT

In [0]:
test1_timedomain = librosa.istft(test1_complex,hop_length=513, win_length=1024,window="hann")

# Write out the cleanes speech signal

In [0]:
librosa.output.write_wav('cleaned_test1.wav', test1_timedomain, sr_test)

In [74]:
import IPython.display as ipd
ipd.Audio('cleaned_test1.wav')

In [75]:
ipd.Audio('test_x_01.wav')

# Load test noisy signal test_02_x.wav and feed its magnitude spectra to trained network

In [0]:
test2, sr_test=librosa.load('test_x_02.wav',sr=None)
Test2=librosa.stft(test2, n_fft=1024, hop_length=513,window="hann")
Test2_absolute = np.abs(Test2)

In [0]:
test2_prediction = sess.run(logits,feed_dict={X: Test2_absolute.T})

# Recover complex valued speech spectrogram of cleaned test signal

In [0]:
test2_complex = np.multiply(Test2,test2_prediction.T)

# Recover time domain speech signal by applying inverse-STFT

In [0]:
test2_timedomain = librosa.istft(test2_complex,hop_length=513, win_length=1024,window="hann")

# Write out the cleanes speech signal

In [0]:
librosa.output.write_wav('cleaned_test2.wav', test2_timedomain, sr_test)

In [86]:
import IPython.display as ipd
ipd.Audio('cleaned_test2.wav')

In [87]:
ipd.Audio('test_x_02.wav')